In [ ]:
!pip install sentence-transformers==2.7.0 datasets==2.19.0 huggingface_hub==0.23.0 faiss-cpu==1.8.0 -qqq

# **사전 학습된 언어 모델을 불러와 문장 임베딩 모델 만들기**

In [ ]:
from sentence_transformers import SentenceTransformer, models

# 허깅페이스 임베딩 모델 로드
transformer_model = models.Transformer('klue/roberta-base')

# 풀링층 생성
pooling_layer = models.Pooling(
    transformer_model.get_word_embedding_dimension(),
    # 평균 풀링 모드
    pooling_mode_mean_tokens=True
)

embedding_model = SentenceTransformer(modules=[transformer_model, pooling_layer])

# **실습 데이터셋 다운로드 및 확인**

In [ ]:
from datasets import load_dataset

klue_sts_train = load_dataset('klue', 'sts', split='train')
klue_sts_test = load_dataset('klue', 'sts', split='validation')
klue_sts_train[0]

# {'guid': 'klue-sts-v1_train_00000',
#  'source': 'airbnb-rtt',
#  'sentence1': '숙소 위치는 찾기 쉽고 일반적인 한국의 반지하 숙소입니다.',
#  'sentence2': '숙박시설의 위치는 쉽게 찾을 수 있고 한국의 대표적인 반지하 숙박시설입니다.',
#  'labels': {'label': 3.7, 'real-label': 3.714285714285714, 'binary-label': 1}}